# SC-VAR Usage

single cell ATAC-seq data

We assign noncoding SNPs to their target genes based on chromatin accessibility in disease-relevant tissues measured by single-cell ATAC sequence. 
Which can be used to find disease risk genes and pathways.

And by calculating the single-cell chromatin accessibility disease score 

Users can infer cell types involved in complex traits and diseases using single-cell epigenomes.

[ This branch does not rely on fine cell annotations and other Omics data. ]

In [ ]:
!pip install sc-var

In [ ]:
import pandas as pd

def generate_atac_risk_file(overlap_matrix, scemagma_result, output_file=None):
    """
    Generate the risk file (CRE_ID, ZSTAT) required for scRiskDB ATAC-seq analysis.
    
    Parameters
    ----------
    overlap_matrix : pandas.DataFrame
        Output from scv.snp_peak() function. 
        Contains the mapping between SNPs and Peaks.
        
    scemagma_result : str
        Path to the output file from scemagma pipeline gene analysis (the .genes.out file).
        
    output_file : str, optional
        Path to save the resulting CSV file (e.g., 'risk_data.csv').
        
    Returns
    -------
    pandas.DataFrame
        A DataFrame with columns ['CRE_ID', 'ZSTAT']
    """
    

    cmagma_gene = pd.read_csv(scemagma_result, sep='\s+') 
    
    if 'GENE' in cmagma_gene.columns:
        cmagma_gene = cmagma_gene.sort_values('P').drop_duplicates(subset=['GENE'], keep='first')
    
    cmagma_gene_sig = cmagma_gene[cmagma_gene['P'] < 0.05]

    filtered_matrix = overlap_matrix[overlap_matrix['gene'].isin(cmagma_gene_sig['GENE'])] # 或 'symbol'
    
    if filtered_matrix.empty:
        print("Warning: No peaks linked to significant risk genes found.")
        return pd.DataFrame(columns=['CRE_ID', 'ZSTAT'])

    peak_weights = get_peak_weights(filtered_matrix)

    risk_df = peak_weights.copy()

    risk_df = risk_df.rename(columns={
        risk_df.columns[0]: 'CRE_ID', 
        'weight': 'ZSTAT'
    })
    

    risk_df = risk_df[['CRE_ID', 'ZSTAT']]
    

    if output_file:
        risk_df.to_csv(output_file, index=False)
        print(f"Risk file saved to: {output_file}")
        
    return risk_df

def get_peak_weights(overlap_matrix):

    '''
    For each peak, determine the lowest p-value of all SNPs that overlap with it.
    '''

    peak_weights = overlap_matrix.groupby(['chr', 'start', 'end']).agg({'pval': 'min'})
    #if min p is 0, replace it with 1e-10
    peak_weights['pval']=peak_weights['pval'].replace(0,1e-10)
    overlap_matrix['zscore']= np.abs(scipy.stats.norm.ppf(overlap_matrix['pval']/ 2))
    #keep zcore corresponding to min p
    peak_weights=pd.merge(peak_weights,overlap_matrix[['chr','start','end','zscore']],on=['chr','start','end'],how='left')
    peak_weights.reset_index(inplace=True)
    peak_weights.rename({'zscore': 'weight'}, axis=1, inplace=True)
    peak_weights['GENE']=peak_weights['chr'].astype(str)+'-'+peak_weights['start'].astype(str)+'-'+peak_weights['end'].astype(str)

    peak_weights.sort_values(['weight'],ascending=False,inplace=True)
    peak_weights.drop_duplicates(subset=['GENE'],keep='first',inplace=True)
    peak_weights=peak_weights[['GENE','weight']]
    return peak_weights

In [3]:
import scdrs
import scanpy as sc
from anndata import AnnData
from anndata import read_h5ad
from scipy import stats
import pandas as pd
import numpy as np
import sc_var
from sc_var import method as scv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import axes
import pylab
from matplotlib.pyplot import rc_context
import os
import warnings
import matplotlib.pyplot as plt
import matplotlib as mpl
import episcanpy as epi
from cycler import cycler

warnings.filterwarnings("ignore")


## preprocessing data

GWAS SNP data format:

chr pos rsids pval

chr1    14773   rs878915777 0.13



==============

Peak data format:

gene chr start end

TTLL10  chr1    826700	827679

In [ ]:
#GWAS data
snp_list=scv.read_gwas('~/data/SCZ.txt')

#IF only have the cicero_conn and cds fdata
#Use the function help to get the peak-gene conn from cicero_conn and cds fdata
peak_list=scv.get_p2g_conn('~/data/conn_path','~/data/fdata_path')
#Or             
#peak data ( Co-accessibility networks )
peak_list=scv.load_peak_data('~/data/atac_cis_coacess.txt')



Find the snp-peak overlap (Take times if data is large)

!! ATTENTION [choose the right reference genome]

check gencode webset for annotation version ( for now v43 for hg19 v44 for hg38)

In [ ]:
#overlp SNPs and peaks 
overlap_matrix = scv.snp_peak(peak_list,snp_list)
#find snp inside the gene body (load the annotatio file from the original magma pipline)
magma = scv.dmagma('~/data/magma0.genes.annot')
#get gene coordinates
corr = scv.gene_corr('~/data/gencode.v44.annotation.gff3.gz',peak_list)

## Get peak-snp-gene annotation

In [ ]:
#SCZ as example
anno=scv.annotate(overlap_matrix,corr,magma,'Schizophrenia.')

Gene Analysis

In [ ]:
#Gene analysis
#choose the 1000 genome reference panel map your GWAS data
!magma\
    --bfile /g1000_eur/g1000_eur \                     
    --pval scz.txt use='rsids,pval' N=328748 \
    --gene-annot /Schizophrenia.scemagma.genes.annot \                                                   
    --out /output/scemagma


In [ ]:
# You need overlap_matrix
# And From MAGMA,you Got '<Disease>.genes.out'

scriskdb_riskfile=generate_atac_risk_file(
    overlap_matrix = overlap_matrix,
    scemagma_result = 'scemagma.genes.out',
    output_file = 'risk_for_upload.csv'
)

# Output: SCZ_risk_for_upload.csv created.
# Now upload this file to scRiskDB!